In [12]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import base64

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from myAnimalShelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "myUser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'GSLogo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())
image_filename_2 = 'CarmensBuddyFinder.png' # my own logo
encoded_image_2 = base64.b64encode(open(image_filename_2, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Div([
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), 
                style = {'height':'10%', 'width':'25%', 'display':'inline-block', 'float':'right'}),
    ]),
    html.Div([
        html.Img(src='data:image/png;base64,{}'.format(encoded_image_2.decode()), 
                 style = {'height':'10%', 'width':'25%'}),
    ]),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Div(
    
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    
        children = 'Choose a filter:',
        style = {'front-style': 'italic', 'front-weight': 'bold', 'display': 'flex'}
    ),
    html.Div(
        [
            dcc.Dropdown(id = 'my_dropdown', 
                options = [
                    {"label": "Water", "value": 'W'}, 
                    {"label": "Mountain or Wilderness", "value": 'MW'}, 
                    {"label": "Disaster or Individual Tracking", "value": 'DIT'}, 
                    {"label": "Reset", "value": 'R'}
                ],
                disabled = False,
                clearable = False,
            ),
            html.Div(id = 'dd-output-container'),
        ], style = {'width': '30%'}, 
    ),

    html.Hr(),
    dt.DataTable(
        id = 'datatable-id',
        columns = [
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data = df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        editable = False, 
        filter_action = "custom", 
        sort_mode = "multi", 
        column_selectable = False, 
        row_selectable = 'multi', 
        row_deletable = False, 
        selected_columns = [], 
        selected_rows = [], 
        page_action = "native", 
        page_current = 0, 
        page_size = 10,
    ),
    html.Br(),
    html.Hr(),
    
# #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            dcc.Graph(
                id='graph-id',
                className='col s12 m6'),
            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

# #############################################
# # Interaction Between Components / Controller
# #############################################



    
@app.callback(
    [Output('datatable-id','data'),
    Output('datatable-id','columns')],
    [Input('my_dropdown', 'value')]
 )
def update_dashboard(my_dropdown):
 ### FIX ME Add code to filter interactive data table with MongoDB queries 

    if ('value' == 'W'):
        df1 = pd.Dataframe.from_records(shelter.read({'$and': [ 
            {'$or': [ {'breed':'Labrador Retriever Mix'}, {'breed':'Chesapeake Bay Retriever'},
                   {'breed':'Newfoundland'}]}, 
            {'sex_upon_outcome':'Intact Female'}, {'age_upon_outcome_in_weeks':{'$gte':26, '$lte':156}}]}))                                                               
                                                                                
    elif ('value' == 'MW'):
        df1 = pd.Dataframe.from_records(shelter.read({'$and': [ 
            {'$or': [ {'breed':'German Shepherd'}, {'breed':'Alaskan Malamute'},
                   {'breed':'Old English Sheepdog'},{'breed':'Siberian Husky'},{'breed':'Rottweiler'}]}, 
            {'sex_upon_outcome':'Intact Male'}, {'age_upon_outcome_in_weeks':{'$gte':26, '$lte':156}}]}))

    elif ('value' == 'DIT'):
        df1 = pd.Dataframe.from_records(shelter.read({'$and': [ 
            {'$or': [ {'breed':'Doberman Pinscher'}, {'breed':'German Sheperd'},
                   {'breed':'Golden Retriever'},{'breed':'Bloodhound'},{'breed':'Rottweiler'}]}, 
            {'sex_upon_outcome':'Intact Male'}, {'age_upon_outcome_in_weeks':{'$gte':20, '$lte':300}}]}))

    elif('value' == 'R'):
        df1 = pd.Dataframe.from_records(shelter.read({}))
        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df1.columns]
    data=df1.to_dict('records')    
        
    return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


@app.callback(
    Output('graph-id', "figure"),
    [Input('datatable-id', "data")]
)
def update_graphs(data):
#     ###FIX ME ####
#     # add code for chart of your choice (e.g. pie chart) #
    df1 = pd.DataFrame.from_dict(data)
    fig = px.pie(df1, names = 'breed',)
    
    return fig


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")]
)
def update_map(viewData):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)

    # Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style = {'width':'1000px', 'height':'500px'}, center = [30.75, -97.48], zoom = 10, children = [
            dl.TileLayer(id = "base-layer-id"), 
            # Marker with tool tip and popup
            dl.Marker(position = [30.75, -97.48], children = [
                dl.Tooltip(dff.iloc[0,4]), 
                dl.Popup([
                    html.H1("Animal Name"), 
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]

app